# Import Sequential dataset

In [1]:
import pandas as pd

data = pd.read_csv("100_Unique_QA_Dataset.csv")
data.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


# Tokenize

In [5]:
def tokenize(text):
    text = text.lower()
    text = text.replace("?","")
    text = text.replace("'","")
    return text.split()

In [6]:
tokenize("What is the capital of France?")

['what', 'is', 'the', 'capital', 'of', 'france']

# vocab

In [7]:
vocab = {"<UKN>":0}

In [8]:
def build_vocab(row):
    tokenized_question = tokenize(row["question"])
    tokenized_answer = tokenize(row["answer"])

    merged_tokens = tokenized_question + tokenized_answer

    for token in merged_tokens:
        if token not in vocab:
            vocab[token] = len(vocab)

In [9]:
data.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [10]:
vocab

{'<UKN>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [12]:
len(vocab)

324

# Convert Word To Numerical Indices

In [13]:
def text_to_indeces(text,vocab):
    indexed_text=[]
    for token in tokenize(text):
        if token in vocab:
            indexed_text.append(vocab[token])
        else:
            indexed_text.append(vocab["<UKN>"])
    return indexed_text

In [15]:
text_to_indeces("What is the boiling point of water in Celsius?",vocab)

[1, 2, 3, 24, 25, 5, 26, 19, 27]

In [16]:
import torch
from torch.utils.data import Dataset, DataLoader

In [17]:
class QAdataset(Dataset):
    def __init__(self,df,vocab):
        self.df = df
        self.vocab = vocab
    def __len__(self):
        return self.df.shape[0]
    def __getitem__(self, index):
        numerical_question = text_to_indeces(self.df.iloc[index]["question"],self.vocab)
        numerical_answer = text_to_indeces(self.df.iloc[index]["answer"],self.vocab)

        return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [18]:
dataset = QAdataset(data,vocab)

In [19]:
dataset[1]

(tensor([1, 2, 3, 4, 5, 8]), tensor([9]))

In [21]:
dataloasder = DataLoader(dataset, batch_size=1, shuffle=True)

# making RNN Model

In [22]:
import torch.nn as nn

In [29]:
class SimpleRNN(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size,embedding_dim=50)
        self.RNN = nn.RNN(50,64,batch_first=True)
        self.fc = nn.Linear(64,vocab_size)
        
    def forward(self,question):
        embedding_question = self.embedding(question)
        hidden, final = self.RNN(embedding_question)
        output = self.fc(final.squeeze(0))

        return output

In [30]:
Learning_rate = 0.001
epochs = 20

In [31]:
model = SimpleRNN(len(vocab))

In [32]:
Lossfunction = nn.CrossEntropyLoss()
optimzer = torch.optim.Adam(model.parameters(), lr = Learning_rate)

# traning loop

In [33]:
for epoch in range(epochs):
    total_epoch_loss = 0
    for question,answer in dataloader:
        
        # forword pass
        outputs = model(question)
        
        # calcluate loss
        loss = Lossfunction(outputs, answer[0])
        
        # back prop
        optimzer.zero_grad()
        loss.backward()
        
        # upgrade grads
        optimzer.step()

        total_epoch_loss = total_epoch_loss + loss.item()
    # priniting the avrage loss per epoch
    avg_loss = total_epoch_loss/len(dataloader)
    print(f"Epoch: {epoch+1} , Loss: {avg_loss} , total_epoch_loss:{total_epoch_loss}" )

Epoch: 1 , Loss: 5.86321685579088 , total_epoch_loss:527.6895170211792
Epoch: 2 , Loss: 5.145487308502197 , total_epoch_loss:463.09385776519775
Epoch: 3 , Loss: 4.31343506442176 , total_epoch_loss:388.2091557979584
Epoch: 4 , Loss: 3.6112950457466972 , total_epoch_loss:325.01655411720276
Epoch: 5 , Loss: 3.0176745229297213 , total_epoch_loss:271.5907070636749
Epoch: 6 , Loss: 2.473293720351325 , total_epoch_loss:222.59643483161926
Epoch: 7 , Loss: 1.9613465944925943 , total_epoch_loss:176.5211935043335
Epoch: 8 , Loss: 1.5216200596756406 , total_epoch_loss:136.94580537080765
Epoch: 9 , Loss: 1.1605187389585707 , total_epoch_loss:104.44668650627136
Epoch: 10 , Loss: 0.8804084440072377 , total_epoch_loss:79.2367599606514
Epoch: 11 , Loss: 0.671313097079595 , total_epoch_loss:60.418178737163544
Epoch: 12 , Loss: 0.5215747036867672 , total_epoch_loss:46.941723331809044
Epoch: 13 , Loss: 0.40984146330091686 , total_epoch_loss:36.88573169708252
Epoch: 14 , Loss: 0.3300449719031652 , total_ep

In [37]:
def predict(model,question,threshold = 0.5):
    num_question = text_to_indeces(question,vocab)

    question_tensor = torch.tensor(num_question).unsqueeze(0)

    output = model(question_tensor)

    probs =torch.nn.functional.softmax(output)

    value, index = torch.max(probs,dim=1)

    if value<threshold:
        print("I Don't Know")
    else:
        print(list(vocab.keys())[index])

In [40]:
predict(model,"What is the boiling point of water in Celsius")

100


C:\Users\updes\AppData\Local\Temp\ipykernel_4164\625736601.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs =torch.nn.functional.softmax(output)
